## Homework 04
  
The [Conversation AI](https://conversationai.github.io/) team, a research initiative founded by [Jigsaw](https://jigsaw.google.com/) and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion).   
  
Kaggle are currently hosting their [second competition](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge#description) on this research. The challenge is to create a model that is capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. The competitions use a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

We shall be using this dataset to benchmark a number of ML models. While the focus of the current competition is to mitigate bias, we will not be using the metric used in the competition. Instead we will be focusing on a simpler metric [Area under the Curve (or AUC)](https://www.kaggle.com/learn-forum/53782) which is suitable to unbalanced binary datasets. Also, we shall not consider different levels of Toxicity; we shall purely take anything marked over the 0.5 level in the measured toxicity range as toxic, and anything underneath as non-toxic. 

We have created a jupyter notbook with some of the tools to model this problem in Deep Learning, using Logistic regression and MLP. Your challenge will be to fill in the models and benchmark the AUC you achieve on these models.

We shall be using the keras deep learning package. As you may know, this is an API into DL frameworks, but is most commonly backed by Tensorflow. [keras.io](keras.io) is a great source for documentation and examples on layers available andn functionality. 

**Have fun!!**


*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

### Set up packages

In [1]:
import os
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


### Load data
Let's load our data and take a peak.   
The competition metrics and fields have too much detail too cover here.   
We will just focus on the comment of the users and whether it was deemed toxic (target>0.5) or not.    

In [2]:
# Lets look at what files we have available. 
os.listdir()

['.ipynb_checkpoints', 'w251_homework04.ipynb', 'v2', 'train.csv.zip']

In [3]:
train_df = pd.read_csv('train.csv.zip', 
                       dtype={'comment_text':str},
                       usecols=['comment_text', 'target'],
                       compression = 'zip')
train_df['target'] = (train_df['target']>0.5).astype(int)
print("Dataframe Number of (rows, columns) : {}".format(train_df.shape))

Dataframe Number of (rows, columns) : (1804874, 2)


In [4]:
pd.options.display.max_colwidth = 200
train_df.head(10)

,target,comment_text
0,0,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"
1,0,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!"
2,0,This is such an urgent design problem; kudos to you for taking it on. Very impressive!
3,0,Is this something I'll be able to install on my site? When will you be releasing it?
4,1,haha you guys are a bunch of losers.
5,1,ur a sh*tty comment.
6,0,hahahahahahahahhha suck it.
7,0,FFFFUUUUUUUUUUUUUUU
8,0,The ranchers seem motivated by mostly by greed; no one should have the right to allow their animals destroy public land.
9,0,It was a great show. Not a combo I'd of expected to be good together but it was.


### Create validation data set
Lets randomly 66/33 split the data into a training and validation set.   
<font color='blue'>**No change needed here - note, please do not change the KFold split parameters, keeping it consistent will help us debug.**</font>

In [5]:
cv = KFold(n_splits=3, shuffle=True, random_state=42)
trn_ids, val_ids = next(cv.split(train_df))
x_train, x_valid = train_df['comment_text'][trn_ids], train_df['comment_text'][val_ids]
y_train, y_valid = train_df['target'].values[trn_ids], train_df['target'].values[val_ids]

### Vectorize Count of tokens

![Count Vectorizer](https://cdn-images-1.medium.com/max/1600/1*LD5XaHzOnoniU4p137FL5g.jpeg)  
We shall start off performing some CPU based Deep Learning operations. Sparse matrices are better run on CPU.   
Do not underestimate CPU based Deep Learning such as MLP; this can be very powerful and outperform complex much more complex DL models.   
Here we create a sparse matrix from the text, with 200K of the most common unigram and bigrams.  
<font color='blue'>
**You task here is to convert the collection of text documents (found in the `comment_text` field) to a matrix of token counts.  
This can be done using the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) in scikit_learn.  
After creating the vecotrizer, fit it based on the train matrix `x_train` and use this vectorizer to transform both the `x_train` and `x_valid` sets.   
Create sparse matrices called `X_trn_mat` and `X_val_mat`, and please call your vectorizer: `vectorizer`.  
Use the parameters max features = 200000 and the token pattern `\w+`. This token pattern matches one or more word characters (same as `[a-zA-Z0-9_]`) only. all other characters are stripped.
Also, we would like to count both unigrams and bigrams (pairs of words), so set the ngram range to `(1,2)`**
    </font>

In [9]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Create a CountVectorizer, called `vectorizer`
# And create sparse matrices X_trn_mat & X_val_mat
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
vectorizer = CountVectorizer(max_features=200000, token_pattern='\w+')
x_fit = vectorizer.fit(x_train)
X_trn_mat = x_fit.transform(x_train)
X_val_mat = x_fit.transform(x_valid)

In [10]:
print(vectorizer.get_feature_names()[:10])
print(vectorizer.get_feature_names()[100000:100000+10])

['0', '00', '000', '0000', '00001', '00004', '00005', '0001', '0002', '00025']
['mangle', 'mangled', 'mangles', 'mangling', 'mangnall', 'mango', 'mangoes', 'mangos', 'mangosteen', 'mangrove']


In [11]:
X_trn_mat

<1203249x200000 sparse matrix of type '<class 'numpy.int64'>'
	with 48337113 stored elements in Compressed Sparse Row format>

### Logistic Regression

![Logistic Regression](https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg)
  
Lets start off with a simple Logistic Regression, which is the very basic [sigmoid activation function](https://en.wikipedia.org/wiki/Sigmoid_function) used in DL.  
Notice we have no hidden layers, we take as input the whole sparse matrix, and as output the binary classifier prediction (`0<=output<=1`).  
The model has 200001 parameters. One coefficient per column in the sparse matrx, plus one bias variable - each of which is learned using gradient descent. 


In [12]:
model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
out = keras.layers.Dense(1, activation='sigmoid')(model_in)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-2))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 200001    
Total params: 200,001
Trainable params: 200,001
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(X_trn_mat, y_train, epochs=2, batch_size=2**13, validation_data=(X_val_mat, y_valid))

Instructions for updating:
Use tf.cast instead.
Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 10s 8us/step - loss: 0.2532 - val_loss: 0.1876
Epoch 2/2
1203249/1203249 [==============================] - 8s 7us/step - loss: 0.1554 - val_loss: 0.1485


In [14]:
preds_lr = model.predict(X_val_mat).flatten()
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_lr)))

AUC score : 0.89556


Looks at the coefficients to see which words are driving toxic and non-toxic sentences. 

In [15]:
feats = np.array(vectorizer.get_feature_names())
importance_index = model.get_weights()[0].flatten().argsort()
print('Top toxic tokens : \n{}'.format(feats[importance_index[-10:]].tolist()))
print('\nTop non-toxic tokens : \n{}'.format(feats[importance_index[:10]].tolist()))

Top toxic tokens : 
['dumb', 'morons', 'pathetic', 'hypocrite', 'crap', 'idiotic', 'stupidity', 'idiot', 'idiots', 'stupid']

Top non-toxic tokens : 
['amen', 'buggy', 'underscores', 'bingo', 'comcast', 'forego', 'bravo', 'sinned', 'hurdle', 'backcountry']


### MLP

![MLP](https://www.researchgate.net/profile/Mouhammd_Alkasassbeh/publication/309592737/figure/fig2/AS:423712664100865@1478032379613/MultiLayer-Perceptron-MLP-sturcture-334-MultiLayer-Perceptron-Classifier-MultiLayer.png)

Here we shall create a Multi layer perceptron. Although relatively simple, these can be very poswerful models and useful when compute power is low. 
<font color='blue'>**Please add three hidden layers to the network using a `relu` activation function.  
You can refer to this [script](https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s) which was an MLP taking first place to win the *Mercari Price Suggestion Challenge*. However note, you can do this task by only adding three or four lines. You should a large increace in performance over the Logistic Regression.**</font>  
Never underestimate the power of an MLP!!

In [103]:
model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Adding three hidden layers and the output layer. 
layer1 = keras.layers.Dense(32, activation='relu')(model_in)
layer2 = keras.layers.Dense(8, activation='relu')(layer1)
layer3 = keras.layers.Dense(4, activation='relu')(layer2)
out = keras.layers.Dense(1)(layer3)
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        (None, 200000)            0         
_________________________________________________________________
dense_93 (Dense)             (None, 32)                6400032   
_________________________________________________________________
dense_94 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_95 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_96 (Dense)             (None, 1)                 5         
Total params: 6,400,337
Trainable params: 6,400,337
Non-trainable params: 0
_________________________________________________________________


In [104]:
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
preds_mlp = model.predict(X_val_mat).flatten()

Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 34s 28us/step - loss: 0.1714 - val_loss: 0.1403
Epoch 2/2
1203249/1203249 [==============================] - 31s 26us/step - loss: 0.1156 - val_loss: 0.1284


In [105]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))

AUC score : 0.92886


### MLP with regularisation

Now lets try regularisation.  
<font color='blue'>**Copy the above MLP model and create a new one adding regularisation.  
    Add l2 regularisation to each of the dense layers. Check on [keras.io](keras.io) to find details on how to add l2 regularisation. Play are around with different level of regularisation to see when you achieve optimal results.   
Generally it is good to choose parameters like this by using different factors of `10`.  
Can you improve on you previous AUC results by using reglarisation?**</font>

In [106]:
from tensorflow.keras import regularizers

model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Adding three hidden layers and the output layer with L2 regularization. 
layer1 = keras.layers.Dense(32, kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-4),
                         bias_regularizer=regularizers.l2(1e-4),
                         activity_regularizer=regularizers.l2(1e-5), 
                         activation='relu')(model_in)
layer2 = keras.layers.Dense(8, kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-4),
                         bias_regularizer=regularizers.l2(1e-4),
                         activity_regularizer=regularizers.l2(1e-5), 
                         activation='relu')(layer1)
layer3 = keras.layers.Dense(4, kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-4),
                         bias_regularizer=regularizers.l2(1e-4),
                         activity_regularizer=regularizers.l2(1e-5))(layer2)
out = keras.layers.Dense(1)(layer3)
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 200000)            0         
_________________________________________________________________
dense_97 (Dense)             (None, 32)                6400032   
_________________________________________________________________
dense_98 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_99 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_100 (Dense)            (None, 1)                 5         
Total params: 6,400,337
Trainable params: 6,400,337
Non-trainable params: 0
_________________________________________________________________


In [107]:
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
preds_mlp = model.predict(X_val_mat).flatten()

Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 37s 30us/step - loss: 0.1697 - val_loss: 0.1444
Epoch 2/2
1203249/1203249 [==============================] - 32s 27us/step - loss: 0.1259 - val_loss: 0.1386


In [108]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))

AUC score : 0.93710
